# Query Cell Types Ontology

# Context

- This notebook has been put together for the MMB demo on 2022-05-30: see [slides](https://docs.google.com/presentation/d/1Ib1_8byK0hVuNS-wPbqmeL5Lcf67m_oegC3-czPw5ws/edit#slide=id.g116ba5ed71e_0_8)
- It has been revised following feedback from the meeting held on 2022-07-07: see [slides](https://docs.google.com/presentation/d/1mgCyYjHerLJLV79GM0kqp3_Htxmru_7QGIr5elUETC0/edit#slide=id.g13b4a370a10_0_19) and [JIRA ticket](https://bbpteam.epfl.ch/project/issues/browse/DKE-942)

## Imports

In [507]:
import json
import rdflib
import getpass
import pandas as pd
from rdflib import RDF, RDFS, XSD, OWL, URIRef, BNode, SKOS
import pprint
from kgforge.core import KnowledgeGraphForge

## Setup

In [ ]:
TOKEN = getpass.getpass()

The cell type ontology is stored in the `neurosciencegraph/datamodels` bucket in the knowledge graph. This is the `bucket` we target with the forge instance below.

In [590]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="neurosciencegraph/datamodels")

In [591]:
forge_atlas = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml",
                            token=TOKEN,
                            # endpoint="https://staging.nise.bbp.epfl.ch/nexus/v1",
                            bucket="bbp/atlas")

## Ontologies

### Set brain region

During the meeting on `2022-05-30`, it was specified that a brain region will serve as entry point when searching for cell types in the MMB context. Hence, this notebook starts by defining a brain region one wants to get cell types for. Since the most complete cell type information is available for the `Cerebral cortex`, this has been set as the default below.

In [592]:
BRAIN_REGION = "Cerebral cortex"
# BRAIN_REGION = "Cerebellum"
# BRAIN_REGION = "Somatosensory areas"

Get brain region id

In [593]:
r = forge.search({"label": BRAIN_REGION})

In [594]:
brain_region = r[0].id

In [595]:
brain_region

'http://api.brain-map.org/api/v2/data/Structure/688'

## Queries

### Get brain regions which do have neuron t-types available

This query will list brain region labels for which the knowledge graph has neuron t-types available

In [596]:
query = f"""

SELECT ?brain_region

WHERE {{
        ?t_type_id subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> ;
                  canHaveBrainRegion ?brain_region_id .        
        ?brain_region_id label ?brain_region .
}}
"""

In [597]:
resources = forge.sparql(query, limit=1000)

In [598]:
df = forge.as_dataframe(resources)

In [599]:
set(df.brain_region)

{'Agranular insular area',
 'Anterior cingulate area',
 'Area prostriata',
 'Cerebellum',
 'Cerebral cortex',
 'Entorhinal area',
 'Entorhinal area, lateral part',
 'Entorhinal area, medial part, dorsal zone',
 'Fasciola cinerea',
 'Field CA1',
 'Field CA2',
 'Field CA3',
 'Hippocampal formation',
 'Hippocampo-amygdalar transition area',
 'Hypothalamus',
 'Induseum griseum',
 'Isocortex',
 'Parasubiculum',
 'Postsubiculum',
 'Presubiculum',
 'Prosubiculum',
 'Retrohippocampal region',
 'Retrosplenial area',
 'Retrosplenial area, ventral part',
 'Subiculum'}

### Get possible t-types for a given brain region

This query lists t-types for a given brain region (i.e. the `BRAIN_REGION` specified above). For demonstration purposes, the `limit` parameter on the query has been set to `100`. This can be increased to get all available t-types. E.g. the total number of available t-types for `Cerebral cortex` on `2022-07-08` was `252`.

In [600]:
query = f"""

SELECT ?brain_region ?t_type

WHERE {{
        ?t_type_id label ?t_type ;
            subClassOf* <https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType> ;
            canHaveBrainRegion <{brain_region}> .        
        <{brain_region}> label ?brain_region . 
}}
"""

In [601]:
resources = forge.sparql(query, limit=1000)

In [602]:
df = forge.as_dataframe(resources)

In [603]:
df.head()

,brain_region,t_type
0,Cerebral cortex,72_Sst
1,Cerebral cortex,73_Sst
2,Cerebral cortex,74_Sst
3,Cerebral cortex,75_Sst
4,Cerebral cortex,76_Sst


### Get possible met-type combinations plus excitatory/inhibitory category for a given brain region

This query returns possible met-type combinations together with the excitatory/inhibitory categories for the brain region one has set above. For each m- e and t- and transmitter-type, the identifier and the current version in the knowledge graph are also being returned. For a simplified view, please run the `df.drop()` cell below. It will only show the labels of a given type. The `version` indicates the revision of a given type in the knowledge graph and has been included following the Cell Types Meeting on 2022-07-07 to help with reproducibility (see also this JIRA ticket: [DKE-942](https://bbpteam.epfl.ch/project/issues/browse/DKE-942)).

In [604]:
query = f"""

SELECT ?brain_region ?brain_region_version ?transmitter ?transmitter_id ?transmitter_version ?t_type ?t_type_id ?t_type_version ?m_type ?m_type_id ?m_type_version ?e_type ?e_type_id ?e_type_version

WHERE {{
        ?t_type_id label ?t_type ;
                canHaveBrainRegion <{brain_region}> ;
                _rev ?t_type_version .
        
        <{brain_region}> label ?brain_region ;
            _rev ?brain_region_version .
        
        ?m_type_id label ?m_type ;
            _rev ?m_type_version ;
            subClassOf* MType ;
            canHaveTType ?t_type_id ;
            subClassOf* / hasNeurotransmitterType ?transmitter_id .
        
        ?transmitter_id label ?transmitter ;        
            _rev ?transmitter_version .

        ?e_type_id label ?e_type ;
            _rev ?e_type_version ;
            subClassOf* EType ;
            subClassOf* / canHaveMType ?m_type_id ;
            subClassOf* / canHaveTType ?t_type_id .            
}}
"""

In [605]:
resources = forge.sparql(query, limit=1000)

In [606]:
df = forge.as_dataframe(resources)

In [607]:
df.head()

,brain_region,brain_region_version,e_type,e_type_id,e_type_version,m_type,m_type_id,m_type_version,t_type,t_type_id,t_type_version,transmitter,transmitter_id,transmitter_version
0,Cerebral cortex,34,cAC,http://uri.interlex.org/base/ilx_0738197,64,L6_MC,http://uri.interlex.org/base/ilx_0381374,68,10_Lamp5,https://bbp.epfl.ch/ontologies/core/ttypes/10_...,45,Inhibitory,https://bbp.epfl.ch/ontologies/core/bmo/Inhibi...,21
1,Cerebral cortex,34,cAC,http://uri.interlex.org/base/ilx_0738197,64,L6_MC,http://uri.interlex.org/base/ilx_0381374,68,100_Sst,https://bbp.epfl.ch/ontologies/core/ttypes/100...,45,Inhibitory,https://bbp.epfl.ch/ontologies/core/bmo/Inhibi...,21
2,Cerebral cortex,34,cAC,http://uri.interlex.org/base/ilx_0738197,64,L6_MC,http://uri.interlex.org/base/ilx_0381374,68,101_Sst,https://bbp.epfl.ch/ontologies/core/ttypes/101...,45,Inhibitory,https://bbp.epfl.ch/ontologies/core/bmo/Inhibi...,21
3,Cerebral cortex,34,cAC,http://uri.interlex.org/base/ilx_0738197,64,L6_MC,http://uri.interlex.org/base/ilx_0381374,68,108_Pvalb,https://bbp.epfl.ch/ontologies/core/ttypes/108...,45,Inhibitory,https://bbp.epfl.ch/ontologies/core/bmo/Inhibi...,21
4,Cerebral cortex,34,cAC,http://uri.interlex.org/base/ilx_0738197,64,L5_MC,http://uri.interlex.org/base/ilx_0381369,68,89_Sst,https://bbp.epfl.ch/ontologies/core/ttypes/89_Sst,44,Inhibitory,https://bbp.epfl.ch/ontologies/core/bmo/Inhibi...,21


In [608]:
df.drop(["brain_region_version", "e_type_id", "e_type_version", "m_type_id", "m_type_version", "t_type_id", "t_type_version", "transmitter_id", "transmitter_version"], axis=1)

,brain_region,e_type,m_type,t_type,transmitter
0,Cerebral cortex,cAC,L6_MC,10_Lamp5,Inhibitory
1,Cerebral cortex,cAC,L6_MC,100_Sst,Inhibitory
2,Cerebral cortex,cAC,L6_MC,101_Sst,Inhibitory
3,Cerebral cortex,cAC,L6_MC,108_Pvalb,Inhibitory
4,Cerebral cortex,cAC,L5_MC,89_Sst,Inhibitory
...,...,...,...,...,...
995,Cerebral cortex,cAC,L6_MC,79_Sst,Inhibitory
996,Cerebral cortex,cAC,L6_MC,8_Lamp5 Lhx6,Inhibitory
997,Cerebral cortex,cAC,L6_MC,80_Sst,Inhibitory
998,Cerebral cortex,cAC,L6_MC,81_Sst,Inhibitory


### Get possible t-types for a given brain region and all the brain regions which are part of that brain region

This query returns possible t-types for the brain region one has set above and all the brain regions that are part of that brain region. E.g. if one specifies `Cerebral cortex` as brain region, this query would return t-types from the `Cerebral cortex` but also t-types for the `Isocortex` or the `Hippocampal formation` since they are both part of the `Cerebral cortex`.

In [609]:
query = f"""

SELECT ?brain_region ?brain_region_version ?t_type ?t_type_id ?t_type_version

WHERE {{
        ?t_type_id label ?t_type ;
                canHaveBrainRegion ?brain_region_id ;
                _rev ?t_type_version .
        
        ?brain_region_id label ?brain_region ;
            ^hasPart* <{brain_region}> ;
            _rev ?brain_region_version .            
}}
"""

In [610]:
resources = forge.sparql(query, limit=500)

In [611]:
df = forge.as_dataframe(resources)

In [612]:
len(set(df.t_type))

387

In [613]:
df

,brain_region,brain_region_version,t_type,t_type_id,t_type_version
0,Field CA1,34,329_CA1-ProS,https://bbp.epfl.ch/ontologies/core/ttypes/329...,44
1,Field CA1,34,330_CA1-ProS,https://bbp.epfl.ch/ontologies/core/ttypes/330...,44
2,Field CA1,34,331_CA1-ProS,https://bbp.epfl.ch/ontologies/core/ttypes/331...,44
3,Field CA1,34,332_CA1-ProS,https://bbp.epfl.ch/ontologies/core/ttypes/332...,44
4,Field CA1,34,333_CA1-ProS,https://bbp.epfl.ch/ontologies/core/ttypes/333...,44
...,...,...,...,...,...
415,Dentate gyrus,34,363_DG,https://bbp.epfl.ch/ontologies/core/ttypes/363_DG,44
416,Dentate gyrus,34,364_DG,https://bbp.epfl.ch/ontologies/core/ttypes/364_DG,44
417,Retrohippocampal region,34,308_L6b RHP,https://bbp.epfl.ch/ontologies/core/ttypes/308...,44
418,Retrohippocampal region,34,309_L6b RHP,https://bbp.epfl.ch/ontologies/core/ttypes/309...,44


### Get m-types together with their transmitter type (sClass)

In [614]:
query = f"""

SELECT ?transmitter ?m_type

WHERE {{
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            subClassOf* / hasNeurotransmitterType / label ?transmitter .           
}}
"""

In [615]:
resources = forge.sparql(query, limit=100)

In [616]:
df = forge.as_dataframe(resources)

In [617]:
df.head()

,m_type,transmitter
0,L3_TPC:C,Excitatory
1,L5_TPC:B,Excitatory
2,L5_TPC:A,Excitatory
3,L3_TPC:A,Excitatory
4,L2_TPC:B,Excitatory


### Get m-types of pyramidal cells (mClass)

In [618]:
query = f"""

SELECT ?mClass ?m_type

WHERE {{
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            subClassOf* <https://neuroshapes.org/PyramidalNeuron> .           
}}
"""

In [619]:
resources = forge.sparql(query, limit=100)

In [620]:
df = forge.as_dataframe(resources)

In [621]:
df.head()

,m_type
0,L3_TPC:C
1,L5_TPC:B
2,L5_TPC:A
3,L3_TPC:A
4,L2_TPC:B


### Get m-types of interneurons (mClass)

TODO: While we do have interneuron as a class, the subclass relationship remains to be added.

In [622]:
query = f"""

SELECT ?mClass ?m_type

WHERE {{
        ?m_type_id label ?m_type ;
            subClassOf* MType ;
            subClassOf* <https://neuroshapes.org/Interneuron> .           
}}
"""

In [623]:
resources = forge.sparql(query, limit=100)

In [624]:
df = forge.as_dataframe(resources)

In [625]:
df.head()

""


### Get m-types with a given morphology and the morphology definition

This query returns m-types which have a given morphological shape. The cell morphologies were taken from the [Phenotype and Trait Ontology](https://ontobee.org/ontology/PATO) (this was done following the request of Georges Khazen who wanted to include the `PATO` deinfitions of morphologies).
Set `MORPHOLOGY` below to one of the following:

- `standard pyramidal morphology`
- `pyramidal family morphology`
- `tufted pyramidal morphology`
- `basket cell morphology`
- `chandelier cell morphology`
- `neurogliaform morphology`
- `Martinotti morphology`
- `cortical bipolar morphology`
- `bitufted cell morphology`

In [626]:
MORPHOLOGY = "basket cell morphology"

In [627]:
query = f"""

SELECT ?cell ?definition

WHERE {{
        ?cell_id subClassOf* / hasMorphologicalPhenotype ?pato_id ;
                  label ?cell .
        ?pato_id subClassOf* / label "{MORPHOLOGY}" .
        ?parent_pato_id label "{MORPHOLOGY}" ;
                <http://purl.obolibrary.org/obo/IAO_0000115> ?definition .
}}
"""

In [628]:
resources = forge.sparql(query, limit=100)

In [629]:
df = forge.as_dataframe(resources)

In [630]:
df.head()

,cell,definition
0,Neocortex Basket Cell,A cell morphology that inheres in multipolar n...
1,Hippocampus basket cell,A cell morphology that inheres in multipolar n...
2,L23_LBC,A cell morphology that inheres in multipolar n...
3,L23_NBC,A cell morphology that inheres in multipolar n...
4,L23_SBC,A cell morphology that inheres in multipolar n...


### Get t-types from a paper specific paper

The [Cell Types and Missing Data - Version 1](https://docs.google.com/spreadsheets/d/1iUgqPszKkYQgkJlmpQSkeyFWcEoOxovsBkoLPtA3qPg/edit#gid=1180597294) spreadsheet which served as source for the `Cell Types Ontology` - lists paper identifiers on the `Notes` sheet. These references were added to the respective t-types.
Set `PAPER` below to one of the following:

- Yao et al. 2021: `https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all`
- Gokce 2016: `https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC5004635/`
- Kozareva et al. 2021: `https://www.nature.com/articles/s41586-021-03220-z`
- Chen et al. 2017: `https://www.sciencedirect.com/science/article/pii/S2211124717303212?via%3Dihub`
- Kalish et al. 2018: `https://www.pnas.org/content/115/5/E1051`

In [632]:
PAPER = "https://www.sciencedirect.com/science/article/pii/S0092867421005018?dgcid=rss_sd_all"

In [633]:
query = f"""

SELECT ?label ?brain_region

WHERE {{
        ?id seeAlso <{PAPER}> ;
            label ?label .
        OPTIONAL {{ ?id canHaveBrainRegion / label ?brain_region }} .
}}
"""

In [634]:
resources = forge.sparql(query, limit=100)

In [635]:
df = forge.as_dataframe(resources)

In [636]:
df.head()

,label,brain_region
0,10_Lamp5,Cerebral cortex
1,100_Sst,Cerebral cortex
2,101_Sst,Cerebral cortex
3,102_Sst HPF,Hippocampal formation
4,103_Sst HPF,Hippocampal formation


### Get the m- e- and t-type placeholders

One of the requirements specified during the meeting on `2022-05-30` was to have a placeholder class for each of the types. We thus implemented an m- e- and t-type placeholder class.

In [637]:
query = """

SELECT ?id ?label

WHERE {
        ?id label ?label .
        FILTER (CONTAINS(STR(?label), 'Placeholder'))
}
"""

In [638]:
resources = forge.sparql(query, limit=100)

In [639]:
df = forge.as_dataframe(resources)

In [640]:
df

,id,label
0,https://bbp.epfl.ch/ontologies/core/bmo/ETypeP...,EType Placeholder
1,https://bbp.epfl.ch/ontologies/core/bmo/MTypeP...,MType Placeholder
2,https://bbp.epfl.ch/ontologies/core/bmo/TTypeP...,TType Placeholder


### Get all cell type combinations and probabilities for a given brain region

This query cell type combinations for a given brain region (i.e. the `BRAIN_REGION` specified above). For demonstration purposes, the `limit` parameter on the query has been set to `1000`. This can be increased to get all available cell type combinations.

In [641]:
query = f"""

SELECT ?brain_region ?m_type ?e_type ?molecular_type ?probability

WHERE {{
        ?probability_id hasTarget / hasSource / hasSomaLocatedIn ?brain_region_id ;
            hasBody / value ?probability ;
            hasTarget ?m_type_target ;
            hasTarget ?e_type_target ;
            hasTarget ?molecular_type_target .
        ?brain_region_id label ?brain_region ;
            ^hasPart* <{brain_region}> .
        ?m_type_target hasSource / a MType ;
            hasSource / label ?m_type .
        ?e_type_target hasSource / a EType ;
            hasSource / label ?e_type .
        ?molecular_type_target hasSource / a NeuronMolecularType ;
            hasSource / label ?molecular_type .
}}
"""

In [642]:
resources = forge.sparql(query, limit=1000)

In [643]:
df = forge.as_dataframe(resources)

In [644]:
df.head()

,brain_region,e_type,m_type,molecular_type,probability
0,Isocortex,bAC,L23_LBC,PV+,0.333333333
1,Isocortex,dNAC,L4_NBC,PV+,0.371428571
2,Isocortex,cAC,L23_LBC,Serpinf1+,0.0
3,Isocortex,cAC,L23_MC,Serpinf1+,0.0
4,Isocortex,cAC,L23_NBC,Serpinf1+,0.0


----

## 2022-08-31 Workshop queries

### Get the atlas hierarchy

In [645]:
atlas_release = forge_atlas.retrieve("https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885", version=1)

In [646]:
atlas_release._store_metadata["_rev"]

1

In [651]:
print(atlas_release)

{
    context: https://bbp.neuroshapes.org
    id: https://bbp.epfl.ch/neurosciencegraph/data/4906ab85-694f-469d-962f-c0174e901885
    type:
    [
        AtlasRelease
        BrainAtlasRelease
    ]
    brainTemplateDataLayer:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/cce06e96-85a4-403d-8b87-634a1554cdb9
        type: BrainTemplateDataLayer
    }
    description: The official Atlas of the Blue Brain Project, derivated from AIBS Mouse CCF v3 (2017)
    name: Blue Brain Atlas
    parcellationOntology:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/0518dd0b-cbc7-43ef-a75f-45631059c8c5
        type: ParcellationOntology
    }
    parcellationVolume:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data/231f6e2e-6366-4ddc-94b6-35ab50c076c0
        type: BrainParcellationDataLayer
    }
    releaseDate:
    {
        type: xsd:date
        @value: 2022-07-13
    }
    spatialReferenceSystem:
    {
        id: https://bbp.epfl.ch/neurosciencegraph/data

In [652]:
ontology = forge_atlas.retrieve(atlas_release.parcellationOntology.id)

In [653]:
forge_atlas.download(ontology, "distribution.contentUrl", ".")

### Get the list of mtypes for each region (region is an input)

#### The below query will get m-types from the specified brain region plus m-types from all child brain regions (this was added during the workshop to illustrate the `down the tree` generalisation idea)

In [717]:
BRAIN_REGION = "Cerebral cortex"

Get brain region id

In [718]:
r = forge.search({"label": BRAIN_REGION})

In [719]:
brain_region = r[0].id

In [720]:
brain_region

'http://api.brain-map.org/api/v2/data/Structure/688'

In [721]:
query = f"""

SELECT ?m_type ?m_type_id ?m_type_version ?brain_region

WHERE {{
        ?m_type_id label ?m_type ;
            _rev ?m_type_version ;
            subClassOf* MType ;
            hasSomaLocatedIn ?brain_region_id .
        
        ?brain_region_id label ?brain_region ;
            ^hasPart* <{brain_region}> ;
            _rev ?brain_region_version . 
}}
"""

In [722]:
resources = forge.sparql(query, limit=1000)

In [723]:
df = forge.as_dataframe(resources)

In [724]:
df.head()

,brain_region,m_type,m_type_id,m_type_version
0,Hippocampal formation,SR_PC,https://bbp.epfl.ch/neurosciencegraph/ontologi...,68
1,Hippocampal formation,GCL_GC,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
2,Hippocampal formation,SL_IS2,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
3,Hippocampal formation,SLM_PPA,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
4,Hippocampal formation,SO_BP,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21


In [725]:
set(df.brain_region)

{'Hippocampal formation', 'Isocortex'}

#### The below query will get m-types from the specified brain region plus m-types from parent brain regions (this was added during the workshop to illustrate the `up the tree` generalisation idea)

In [726]:
BRAIN_REGION = "Entorhinal area, medial part, dorsal zone"

Get brain region id

In [727]:
r = forge.search({"label": BRAIN_REGION})

In [728]:
brain_region = r[0].id

In [729]:
brain_region

'http://api.brain-map.org/api/v2/data/Structure/926'

In [730]:
query = f"""

SELECT ?m_type ?m_type_id ?m_type_version ?brain_region

WHERE {{
        ?m_type_id label ?m_type ;
            _rev ?m_type_version ;
            subClassOf* MType ;
            hasSomaLocatedIn ?brain_region_id .
        
        ?brain_region_id label ?brain_region ;
            hasPart* <{brain_region}> ;
            _rev ?brain_region_version . 
}}
"""

In [731]:
resources = forge.sparql(query, limit=1000)

In [732]:
df = forge.as_dataframe(resources)

In [733]:
df.head()

,brain_region,m_type,m_type_id,m_type_version
0,Hippocampal formation,SR_PC,https://bbp.epfl.ch/neurosciencegraph/ontologi...,68
1,Hippocampal formation,GCL_GC,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
2,Hippocampal formation,SL_IS2,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
3,Hippocampal formation,SLM_PPA,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21
4,Hippocampal formation,SO_BP,http://bbp.epfl.ch/neurosciencegraph/ontologie...,21


In [734]:
set(df.brain_region)

{'Hippocampal formation'}

### Get the mtype density nrrd file for each region (region is an input)

In [735]:
# TODO; these data are not integrated in the knowledge graph

### Get the etype ratio for each mtype for each region (region and mtype are inputs)

In [736]:
# TODO; these data are not integrated in the knowledge graph

### Get if the mtype is Excitatory or Inhibitory (mtype is input)

See `Get m-types together with their transmitter type (sClass)` query above

### Get if the mtype is pyramidal or interneuron (mtype is input)

See `Get m-types of pyramidal cells (mClass)` and `Get m-types of interneurons (mClass)` queries above

### Get the mini frequency for the mtype (mtype is input)

In [737]:
# TODO; these data are not yet integrated in the knowledge graph

### Get all brain regions of layer 1 of the neocortex

In [738]:
query = f"""

SELECT ?id ?region

WHERE {{
        ?id hasLayerLocationPhenotype <http://purl.obolibrary.org/obo/UBERON_0005390> ;
            subClassOf* BrainRegion ;
            label ?region
}}
"""

In [739]:
resources = forge.sparql(query, limit=1000)

In [740]:
df = forge.as_dataframe(resources)

In [742]:
df.head()

,id,region
0,http://api.brain-map.org/api/v2/data/Structure...,"Primary somatosensory area, trunk, layer 1"
1,http://api.brain-map.org/api/v2/data/Structure...,"Primary somatosensory area, lower limb, layer 1"
2,http://api.brain-map.org/api/v2/data/Structure...,"Taenia tecta, dorsal part, layer 1"
3,http://api.brain-map.org/api/v2/data/Structure...,"Taenia tecta, ventral part, layer 1"
4,http://api.brain-map.org/api/v2/data/Structure...,"Parasubiculum, layer 1"
